In [1]:
import os
import requests
from openai import OpenAI
from PyPDF2 import PdfReader

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

## Read the text file from a local URL

> As we have 3 transcription text files, a file containing the 3 transcripts has been created.

In [2]:
def read_txt_file(url_local):
    with open(url_local, 'r', encoding='latin-1') as archivo:
        return archivo.read()

In [3]:
transcriptions = read_txt_file('../transcriptions/acento_andaluz.txt')
transcriptions

"Dicen que viajar es lo Ãºnico en lo que gastas dinero, pero que te hace mÃ¡s rico. Pues a mÃ\xad otro dÃ\xada me entrÃ³ el gusanillo este de que tenÃ\xada ganas de salir de Granada a hacer un viajillo y se lo planteÃ© a mi hermana. Ya tengo ganas de que nos caigamos un viajillo. No tengo pasta, tÃ\xado. Pues nos metemos el primer vuelo que pillemos, es lo que te digo, y emplazamos chileras, Â¿quÃ© me dices? Venga, coÃ±o, dale. Â¡Dale! Â¡VÃ¡monos a la obra! Sky Scanner, vuelo mÃ¡s barato, Babum, Babaveiro, 40 euros, Madrid, Amsterdam. Primera parte del viaje. Cogemos las dos catetas, estaciÃ³n de autobuses de Granada a Madrid. Cinco horas. Me toca Pasillo. Cuellazo al canto. Â¿Por quÃ©? Porque nos faltaba lo que viene siendo el posapescuezo este, que le llamo yo. Ahora, pues yo lleguÃ© a Madrid, Â¿sabes lo que te digo? Con el cuello y la espalda que parecÃ\xada que habÃ\xada estado vareando olivos un mes entero. Yo dando cuellazo y mi hermana, que la habÃ\xada tocado a ventanilla, ronc

## Create dataset
This function to generate questions and answers, which will be our dataset

In [5]:
def generate_questions(text, num_questions=20):
    system_content = """
        Eres un experto generando preguntas variadas sobre un texto dado. Todas las preguntas generadas DEBEN SER UNICAS.
        Es muy importante que generes la cantidad de preguntas solicitadas por el usuario.
    """
    user_content = f"""
        Instrucciones:
        1. Genera {num_questions} preguntas sobre el contenido del texto:\n{text}.
        2. Las preguntas deben ser variadas.
        3. Las preguntas deben ser sobre los temas tratados en el texto.
        4. Sólo retorna las preguntas.
        CUMPLE CON LAS INSTRUCCIONES ANTES DE GENERAR LA PREGUNTA.
        RECUERDA QUE SON {num_questions} PREGUNTAS SOBRE SEVILLA.
    """

    questions_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_content,
            },
            {
                "role": "user",
                "content": user_content,
            },
        ],
        temperature=0.7,
        max_tokens=1024,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0
    )

    questions = questions_response.choices[0].message.content.split("\n")
    return questions

In [131]:
questions = generate_questions(text=transcriptions)
questions

['1. ¿Por qué el vídeo de hoy está dedicado a la gente que cree que el universo conspira contra ellos?',
 '2. ¿A qué hora había puesto la alarma la narradora para despertarse?',
 '3. ¿Qué hizo la narradora después de tomarse su café y su cigarro?',
 '4. ¿Qué le pasó a la narradora cuando intentó tirar de la cadena del inodoro?',
 '5. ¿Qué error cometió la narradora al hacer una captura de pantalla del mensaje de un chico que le gustaba?',
 '6. ¿Cómo quedó la pantalla del móvil de la narradora después de caerse al suelo?',
 '7. ¿Por qué no llevaba la narradora una carcasa en su móvil?',
 '8. ¿Qué prenda inapropiada usó la narradora para ir al gimnasio y por qué?',
 '9. ¿Qué problema tuvo la narradora en el barrito donde fue a comer con una amiga?',
 '10. ¿Qué le pasó a la narradora al pisar un mojón de camino a casa?',
 '11. ¿Por qué la narradora tuvo que ducharse con agua fría?',
 '12. ¿Qué situación incómoda vivió la narradora en el baño de casa de unos colegas?',
 '13. ¿Qué apodo le 

In [132]:
def generate_answer_for_each_question(text, questions):
    questions_answer = []
    for question in questions:
        answers_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": """
                        Eres una Sevillana experta en responder preguntas sobre un texto dado.
                    """,
                },
                {
                    "role": "user",
                    "content": f"""
                        Responde la siguiente pregunta basada en el texto manteniendo la personalidad, entonacion, 
                        frases, oraciones, expresiones y palabras andaluzas utilizadas por el interlocutor en nuestro texto:\n\nTexto: {text}, 
                        se conciso y eficaz en la elaboracion de tus respuestas.
                        Responda a la pregunta\n\nPregunta: {question}
                        MUY IMPORTANTE:
                        1. Responde utilizando el acento andaluz, por lo que si es necesario sustituye las palabras por las palabras típicas Andaluzas.
                        2. No inicies todas las respuestas con la misma frase.
                        3. Siempre debes responder como si fueras la persona que habla en el texto.
                    """,
                }
            ],
            temperature=1,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )

        answer = answers_response.choices[0].message.content
        questions_answer.append((question, answer))
        print((question, answer))
    
    return questions_answer

In [4]:
questions_answers = generate_answer_for_each_question(text, questions)

NameError: name 'generate_answer_for_each_question' is not defined

## Save questions and answers in a new text file

In [112]:
with open('./questions_answers.txt', 'w', encoding='utf-8') as file:
    for question, answer in questions_answers:
        print(f"Pregunta: {question}\nRespuesta: {answer}\n\n")
        file.write(f"Pregunta: {question}\nRespuesta: {answer}\n\n")

Pregunta: 1. ¿Cuál es el origen histórico de la Giralda de Sevilla?
Respuesta: ¡Ole, qué preguntita más buena! Mira, la Giralda es uno de los monumentos más emblemá'cos de Sevilla. Su origen viene de la época de los almohades, allá por el siglo XII, cuando era el alminar de la Gran Mezquita. Fue diseñada pa' ser el minarete más alto del mundo islámico, con sus más de 100 metros de altura.

Con la Reconquista, cuando los cristianos tomaron Sevilla, decidieron reutilizar esta maravilla pa' la catedral que estaban levantando. Así que en el siglo XVI, le añadieron el cuerpo de campanas y la famosa estatua del Giraldillo, que gira con el viento y le da el nombre actual.

Así que ya sabes, la Giralda es un mezcla perfecta de la historia musulmana y cristiana de Sevilla, una auténtica joya que no te puedes perder si visitas la ciudad. ¡Un abrazo y disfruta de Sevilla!


Pregunta: 2. ¿Qué platos típicos sevillanos no puedes dejar de probar?
Respuesta: ¡Óle! Los platos que no te puedes perder e